Copyright (c) 2024 Habana Labs, Ltd. an Intel Company.
#### Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License. You may obtain a copy of the License at https://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
# Intel&reg; Gaudi&reg; 2 Model Profiling and Optimization using HuggingFace

## Objective
This tutorial will show the user how to run the Intel Gaudi Profiling tools: the habana_perf_tool and the Tensorboard plug-in on the Intel Gaudi 2 AI Accelerator, and the profiling trace viewer.  These tools will provide the user valueable optimization tips and information to modify any model for better performance.   Following these steps and using these tools can help you better understand some of the bottlenecks of your model.  For more information, please refer to the [Profiling](https://docs.habana.ai/en/latest/Profiling/index.html) section of the documentation for info on how to setup the profiler and the [Optimization Guide](https://docs.habana.ai/en/latest/PyTorch/Model_Optimization_PyTorch/index.html) for additional background on other optimization techniques.

| Task                                 | Description                                             | Details                                         |
|--------------------------------------|---------------------------------------------------------|-------------------------------------------------|
| PyTorch Profiling with TensorBoard   | Obtains Gaudi-specific recommendations for performance using TensorBoard. | [Profiling with PyTorch](https://docs.habana.ai/en/latest/Profiling/Profiling_with_PyTorch.html#profiling-with-pytorch)        |
| Review the PT_HPU_METRICS_FILE      | Looks for excessive re-compilations during runtime.     | [Runtime Environment Variables](https://docs.habana.ai/en/latest/PyTorch/Reference/Runtime_Flags.html#pytorch-runtime-flags)                   |                         
| Profiling Trace Viewer               | Uses Perfetto to view traces.           |  [Getting Started with Intel Gaudi Profiler](https://docs.habana.ai/en/latest/Profiling/Intel_Gaudi_Profiling/Getting_Started_with_Profiler.html#getting-started-with-profiler)                      |                         
| Model Logging                        | Sets ENABLE_CONSOLE to set Logging for debug and analysis. | [Runtime Environment Variables](https://docs.habana.ai/en/latest/PyTorch/Reference/Runtime_Flags.html#pytorch-runtime-flags)                |                         




### Initial Setup
To run the this jupyter notebook and the Tensorboard viewer, set the appropriate ports for access when you ssh into the Intel Gaudi 2 node. you need to ensure that the following ports are open:
* 8888 (for running this jupyter notebook)
* 6006 (for running Tensorboard)    

Do to this, you need to add the following in your overall ssh commmand when connecting to the Intel Gaudi Node:

`ssh -L 8888:localhost:8888 -L 6006:localhost:6006 .... `

We start with an Intel Gaudi PyTorch Docker image and run this notebook.   For this example, we'll be using the [Swin Transformer](https://huggingface.co/microsoft/swin-base-patch4-window7-224-in22k) model from the Hugging Face Repository running on Hugging Face's Optimum-Habana library.  So the first step is to load the Optimum-Habana library and model repository:

In [ ]:
%cd ~
!git clone https://github.com/huggingface/optimum-habana.git
%cd optimum-habana
!git checkout v1.14.0
!pip install optimum-habana==1.14.0
!pip install pickleshare ipython

We now will go into the image-classification task and load the specfic requirements for the task:

In [ ]:
%cd ~/optimum-habana/examples/image-classification
!pip install --quiet -r requirements.txt

### Running the Model
Now that the model is loaded, we'll run the model and look for the trace files for analysis. 

For this model script we can see the profiling set in the utils.py. 
For other models not in optimum-habana, users can refer to [Profiling_with_PyTorch](https://docs.habana.ai/en/latest/Profiling/Profiling_with_PyTorch.html) to setup profiling 

In [4]:
%%sh

cat -n ../../optimum/habana/utils.py | head -n 313 | tail -n 13

   301	            schedule = torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=1)
   302	            activities = [torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.HPU]
   303	
   304	            profiler = torch.profiler.profile(
   305	                schedule=schedule,
   306	                activities=activities,
   307	                on_trace_ready=torch.profiler.tensorboard_trace_handler(output_dir),
   308	                record_shapes=record_shapes,
   309	                with_stack=False,
   310	            )
   311	            self.start = profiler.start
   312	            self.stop = profiler.stop
   313	            self.step = profiler.step


Run Model to collect trace file (unoptimized)
Swin Transformer is a model that capably serves as a general-purpose backbone for computer vision. run_image_classification.py is a script that showcases how to fine-tune Swin Transformer on HPUs.

Notice the torch profiler specific commands:

- `--profiling_warmup_steps 10` - profiler will wait for warmup steps
- `--profiling_steps 3` - records for the next active steps  
                             
The collected trace files will be saved to ./hpu_profile

In [5]:
!python run_image_classification.py \
    --model_name_or_path microsoft/swin-base-patch4-window7-224-in22k \
    --dataset_name cifar10 \
    --output_dir /tmp/outputs/ \
    --remove_unused_columns False \
    --image_column_name img \
    --do_train \
    --learning_rate 3e-5 \
    --num_train_epochs 2 \
    --per_device_train_batch_size 64 \
    --evaluation_strategy no \
    --save_strategy no \
    --load_best_model_at_end False \
    --save_total_limit 3 \
    --seed 1337 \
    --use_habana \
    --use_lazy_mode \
    --use_hpu_graphs_for_training \
    --gaudi_config_name Habana/swin \
    --throughput_warmup_steps 3 \
    --bf16 \
    --report_to none \
    --throughput_warmup_steps 2 \
    --overwrite_output_dir \
    --ignore_mismatched_sizes \
    --profiling_warmup_steps 10 \
    --profiling_steps 3

/root/optimum-habana/optimum/habana/transformers/training_args.py:380: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/lib/python3.10/inspect.py:288: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(object, types.FunctionType)
/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
10/18/2024 20:44:51 - WARNING - __main__ - 

In [8]:
%cd hpu_profile
%ls -al

[Errno 2] No such file or directory: 'hpu_profile'
/root/Gaudi-tutorials/PyTorch/Profiling_and_Optimization/optimum-habana/examples/image-classification/hpu_profile
total 187084
drwxr-xr-x 1 root root         0 Oct 18 20:58 ./
drwxr-xr-x 1 root root         0 Oct 18 20:43 ../
drwxr-xr-x 1 root root         0 Oct 18 20:58 .ipynb_checkpoints/
-rw-r--r-- 1 root root 191570168 Oct 18 20:45 sc09wynn05-hls2_14734.1729284340533778439.pt.trace.json


### Reviewing the Details in Tensorboard and perf_tool
Now that the training is completed, you can see the trace files (...pt.trace.json) have been generated and now can be viewed.  Two types of information are produced by TensorBoard:

Model Performance Tracking - While your workload is being processed in batches, you can track the progress of the training process on the dashboard in real-time by monitoring the model’s cost (loss) and accuracy.

Profiling Analysis - Right after the last requested step was completed, the collected profiling data is analyzed by TensorBoard and then immediately submitted to your browser, without any need to wait till the training process is completed.

In [8]:
%load_ext tensorboard
%tensorboard --logdir=~/optimum-habana/examples/image-classification/hpu_profile --port 6006    # Your port selection may vary, default is 6006

If you do not want to run the TensorBoard UI, you can take the same .json log files and use the habana_perf_tool that will parse the existing .json file and provide the same recommendations for performance enhancements, but in a text form.

In [7]:
!habana_perf_tool --trace /root/optimum-habana/examples/image-classification/hpu_profile/sc09wynn05-hls2_14734.1729284340533778439.pt.trace.json

2024-10-18 20:56:53,275 - pytorch_profiler - DEBUG - Loading /root/Gaudi-tutorials/PyTorch/Profiling_and_Optimization/optimum-habana/examples/image-classification/hpu_profile/sc09wynn05-hls2_14734.1729284340533778439.pt.trace.json
Import Data (KB): 100%|█████████████| 187080/187080 [00:01<00:00, 112314.37it/s]
2024-10-18 20:56:56,310 - pytorch_profiler - DEBUG - Please wait for initialization to finish ...
2024-10-18 20:57:02,010 - pytorch_profiler - DEBUG - PT Track ids: BridgeTrackIds.Result(pt_bridge_launch='51,47,7', pt_bridge_compute='16', pt_mem_copy='7', pt_mem_log='', pt_build_graph='46,49,50,6')
2024-10-18 20:57:02,010 - pytorch_profiler - DEBUG - Track ids: TrackIds.Result(forward='5', backward='45', synapse_launch='0,1,48', synapse_wait='2,10', device_mme='41,42,43,44', device_tpc='17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40', device_dma='8,11,12,13,14,15')
2024-10-18 20:57:02,897 - pytorch_profiler - DEBUG - Device ratio: 37.66 % (193.313984 ms, 

### Using the Perfetto Trace Viewer
Finally, to view the details of the Intel Gaudi Device itself, you can view the traces in the perfetto trace viewer.  

This step requires you to set the `hl-prof-config` settings and the Environment variable `HABANA_PROFILE=1` as shown below, this will generate the .hltv file that can be viewed using https://perfetto.habana.ai.  Since this is using the Gaudi profiler, the runtime profiling commands need to be removed.  At the end of this run, you will see a `my_profiling_session_12345.hltv` file that can be loaded into the Perfetto browser.

For More Information to enable your model to use the Habana Perfetto Trace viewer, you can refer to the documentation https://docs.habana.ai/en/latest/Profiling/Intel_Gaudi_Profiling/Getting_Started_with_Profiler.html

In [9]:
%cd ..
!hl-prof-config -e off -phase=multi-enq -g 1-20 -s my_profiling_session
!export HABANA_PROFILE=1

/root/Gaudi-tutorials/PyTorch/Profiling_and_Optimization/optimum-habana/examples/image-classification
Resetting config file /root/.habana/prof_config.json
Edited file /root/.habana/prof_config.json


In [ ]:
!HABANA_PROFILE=1 python run_image_classification.py \
    --model_name_or_path microsoft/swin-base-patch4-window7-224-in22k \
    --dataset_name cifar10 \
    --output_dir /tmp/outputs/ \
    --remove_unused_columns False \
    --image_column_name img \
    --do_train \
    --learning_rate 3e-5 \
    --num_train_epochs 2 \
    --per_device_train_batch_size 64 \
    --evaluation_strategy no \
    --save_strategy no \
    --load_best_model_at_end False \
    --save_total_limit 3 \
    --seed 1337 \
    --use_habana \
    --use_lazy_mode \
    --report_to none \
    --use_hpu_graphs_for_training \
    --gaudi_config_name Habana/swin \
    --throughput_warmup_steps 3 \
    --bf16 \
    --throughput_warmup_steps 2 \
    --overwrite_output_dir \
    --ignore_mismatched_sizes 
    #--profiling_warmup_steps 10 \
    #--profiling_steps 3

In [12]:
!ls -l *.hltv

-rw-r--r-- 1 root root 58536303 Oct 10 22:04 my_profiling_session_521.hltv


Consult the [Analysis guide](https://docs.habana.ai/en/latest/Profiling/Intel_Gaudi_Profiling/Analysis.html) for performing a thorough analysis of the above .hltv profile.

In [ ]:
exit()